In [25]:
print('Started...')
import glob
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import png
import time
import numpy as np
import os
from fnmatch import fnmatch


# Gobals
DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/dcm/malignant/1.2.840.113681.2230565232.95*'
#DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/dcm/normal/1.2.840.113681.2230565232.95*'
DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/IMAGE_DATABASE/PUBLIC_SHARE/IMAGES/STANDARD_SET/15'

# File walk method

fMal = []
for path, subdirs, files in os.walk(DICOM_FILES):
    for name in files:
        if fnmatch(name, '*.dcm'):
            fMal.append(os.path.join(path, name))
            
print(len(fMal), ' dicom images found')


# Load dicom files into np array
malignant = np.array([])
dicomImg = {'malignant':np.array([]), 'normal':np.array([])}
count = 0
print(len(fMal), ' Malignant found')
print('Loading images...')
for f in fMal:
    dicomImg['malignant'] = np.append(dicomImg['malignant'], pydicom.dcmread(f))
    count += 1
    print(count, '/', len(fMal))


# __________________________________________
print(dicomImg['malignant'].shape)
# print(dicomImg['malignant'][0].shape)
print(dicomImg['malignant'][0])
#print(dicomImg['malignant'])
print('name:\n', dicomImg['malignant'][0].PresentationIntentType)

forProcessingCount = 0
for _ in dicomImg['malignant']:
    tmp = _.PresentationIntentType
    print(tmp)
    if tmp == 'FOR PROCESSING':
        forProcessingCount += 1
print('\nTotal DICOM: ', len(dicomImg['malignant']))
print('Total FOR PROCESSING: ', forProcessingCount)

    









   
# Import xls file
xls = pd.ExcelFile('/vol/vssp/cvpwrkspc01/scratch/wm0015/batch_50_IMAGE.xls')
sheet = xls.parse(1)

# Create a dict where the key is the image name
# Each key has the image, and coords
img = {}
for i in range(len(fMal)):
    key = os.path.basename(fMal[i])[:-4]
    # Find row in the xml file that holds the img info
    try:
        indx = [_==key for _ in sheet['ReferencedSOPInstanceUID']].index(True) # ImageSOPIUID, ReferencedSOPInstanceUID
        img.update({key:{}})
        img[key].update({'img': dicomImg['malignant'][i].pixel_array})
        img[key].update({'x': [sheet['X1'][indx], sheet['X2'][indx]]})
        img[key].update({'y': [sheet['Y1'][indx], sheet['Y2'][indx]]})
    except ValueError:
        print('MY_ERROR: key not found:\n', key)
        
print(len(img), 'DICOM images extracted')
# print(img)
# Crop the images to given ROI
toDelete = [] # Keep track of error causing keys and delete after loop
for key in img:
    try:
        tmp = img[key]['img']
        x = img[key]['x']
        y = img[key]['y']
        x = [int(x[0]), int(x[1])]
        y = [int(y[0]), int(y[1])]
        #x is width, y is height
        #in numpy array, y,x
        img[key].update({'cropROI':tmp[y[0]:y[1], x[0]:x[1]]}) 
    except:
        print('ROI extraction failed...Removing key\nkey:  ',key, '\nx  :', x,'\ny:  ',y)
        toDelete.append(key)
    
#img_calc = img['1.2.840.113681.2230565232.954.3504500766.32'] 
for _ in toDelete:
    img.pop(_)


if 1==2: #Write images to disk with markers and basic crop
    for key in img:
        plt.figure(figsize=(20,20))
        marker = [(img[key]['x'][1] + img[key]['x'][0])/2, (img[key]['y'][1] + img[key]['y'][0])/2 ]   
        plt.imshow(img[key]['img']/16383, cmap='gray', vmin=0, vmax=0.2)
        plt.plot(marker[0], marker[1], marker='x', color=[1,0,1], markersize=30)
        plt.savefig('./marked/normal/' + key +'_full.png')
        #plt.show()
        plt.close()

        plt.figure(figsize=(20,20))
        plt.imshow(img[key]['cropROI'], cmap='gray') 
        plt.savefig('./marked/normal/' + key +'_crop.png')   
        #plt.show()
        plt.close()

# Crop the images so that the ROI is centred but all crops are the same size
if 1 == 1:
    cS = 256
    for key in img:
        tmp = img[key]['img']
        x = img[key]['x']
        y = img[key]['y']
        c = [round((x[0]+x[1])/2), round((y[0]+y[1])/2)]
        # Pad images before cropping (wrap around)
        pad = 1000
        tmp = np.pad(tmp, pad, mode='wrap')
        img[key].update({'crop': tmp[int(c[1]-cS/2+pad):int(c[1]+cS/2+pad), int(c[0]-cS/2+pad):int(c[0]+cS/2+pad)]})
        # Reshape from (256, 256) to (256, 256, 1)
        img[key]['crop'] = np.reshape(img[key]['crop'],(img[key]['crop'].shape[0], img[key]['crop'].shape[1], 1))
    
# Find bit depth
if 1 == 2:
    maxmax = 0
    for key in img:
        tmp = img[key]['img']
        print(np.amax(tmp))
        if np.amax(tmp) > maxmax:
            maxmax = np.amax(tmp)
    print('The largest value is: ', maxmax)
    
# Find average ROI size
if 1 == 2:
    totalX = 0
    totalY = 0
    for key in img:
        x = img[key]['x']
        y = img[key]['y']
        totalX += x[1] - x[0]
        totalY += y[1] - y[0]
    print('Average ROI width: ', totalX/len(img), '\nAverage ROI length: ', totalY/len(img))
    
# View crops / save to disk
if 1 == 2:
    count = 0
    for key in img:
        if count == 0:
            print(img[key]['crop'])

        count+=1
        f = open('./crop256/normal/' + key + '.png', 'wb')
        w = png.Writer(width = cS, height = cS, bitdepth=16, greyscale=True)
        w.write(f, img[key]['crop'])
        f.close()
        print(count, '/', len(img))
        if count == -1:
            break




print('Done')

Started...
1488  Malignant found
1489  Normal found
1 / 1489
2 / 1489
3 / 1489
4 / 1489
5 / 1489
6 / 1489
7 / 1489
8 / 1489
9 / 1489
10 / 1489
11 / 1489
12 / 1489
13 / 1489
14 / 1489
15 / 1489
16 / 1489
17 / 1489
18 / 1489
19 / 1489
20 / 1489
21 / 1489
22 / 1489
23 / 1489
24 / 1489
25 / 1489
26 / 1489
27 / 1489
28 / 1489
29 / 1489
30 / 1489
31 / 1489
32 / 1489
33 / 1489
34 / 1489
35 / 1489
36 / 1489
37 / 1489
38 / 1489
39 / 1489
40 / 1489
41 / 1489
42 / 1489
43 / 1489
44 / 1489
45 / 1489
46 / 1489
47 / 1489
48 / 1489
49 / 1489
50 / 1489
51 / 1489
52 / 1489
53 / 1489
54 / 1489
55 / 1489
56 / 1489
57 / 1489
58 / 1489
59 / 1489
60 / 1489
61 / 1489
62 / 1489
63 / 1489
64 / 1489
65 / 1489
66 / 1489
67 / 1489
68 / 1489
69 / 1489
70 / 1489
71 / 1489
72 / 1489
73 / 1489
74 / 1489
75 / 1489
76 / 1489
77 / 1489
78 / 1489
79 / 1489
80 / 1489
81 / 1489
82 / 1489
83 / 1489
84 / 1489
85 / 1489
86 / 1489
87 / 1489
88 / 1489
89 / 1489
90 / 1489
91 / 1489
92 / 1489
93 / 1489
94 / 1489
95 / 1489
96 / 14

751 / 1489
752 / 1489
753 / 1489
754 / 1489
755 / 1489
756 / 1489
757 / 1489
758 / 1489
759 / 1489
760 / 1489
761 / 1489
762 / 1489
763 / 1489
764 / 1489
765 / 1489
766 / 1489
767 / 1489
768 / 1489
769 / 1489
770 / 1489
771 / 1489
772 / 1489
773 / 1489
774 / 1489
775 / 1489
776 / 1489
777 / 1489
778 / 1489
779 / 1489
780 / 1489
781 / 1489
782 / 1489
783 / 1489
784 / 1489
785 / 1489
786 / 1489
787 / 1489
788 / 1489
789 / 1489
790 / 1489
791 / 1489
792 / 1489
793 / 1489
794 / 1489
795 / 1489
796 / 1489
797 / 1489
798 / 1489
799 / 1489
800 / 1489
801 / 1489
802 / 1489
803 / 1489
804 / 1489
805 / 1489
806 / 1489
807 / 1489
808 / 1489
809 / 1489
810 / 1489
811 / 1489
812 / 1489
813 / 1489
814 / 1489
815 / 1489
816 / 1489
817 / 1489
818 / 1489
819 / 1489
820 / 1489
821 / 1489
822 / 1489
823 / 1489
824 / 1489
825 / 1489
826 / 1489
827 / 1489
828 / 1489
829 / 1489
830 / 1489
831 / 1489
832 / 1489
833 / 1489
834 / 1489
835 / 1489
836 / 1489
837 / 1489
838 / 1489
839 / 1489
840 / 1489
841 / 1489

1455 / 1489
1456 / 1489
1457 / 1489
1458 / 1489
1459 / 1489
1460 / 1489
1461 / 1489
1462 / 1489
1463 / 1489
1464 / 1489
1465 / 1489
1466 / 1489
1467 / 1489
1468 / 1489
1469 / 1489
1470 / 1489
1471 / 1489
1472 / 1489
1473 / 1489
1474 / 1489
1475 / 1489
1476 / 1489
1477 / 1489
1478 / 1489
1479 / 1489
1480 / 1489
1481 / 1489
1482 / 1489
1483 / 1489
1484 / 1489
1485 / 1489
1486 / 1489
1487 / 1489
1488 / 1489
1489 / 1489
1489 DICOM images extracted
ROI extraction failed...Removing key
key:   1.2.840.113681.181135467.1351153030.3804.261 
x  : [nan, nan] 
y:   [nan, nan]
ROI extraction failed...Removing key
key:   1.2.840.113681.181135467.1301645028.2852.197 
x  : [nan, nan] 
y:   [nan, nan]
ROI extraction failed...Removing key
key:   1.2.840.113681.2229461008.958.3554348303.594 
x  : [nan, nan] 
y:   [nan, nan]
ROI extraction failed...Removing key
key:   1.2.840.113619.2.227.20792479062.2077120920135836.89 
x  : [nan, nan] 
y:   [nan, nan]
ROI extraction failed...Removing key
key:   1.2.840.

614 / 1483
615 / 1483
616 / 1483
617 / 1483
618 / 1483
619 / 1483
620 / 1483
621 / 1483
622 / 1483
623 / 1483
624 / 1483
625 / 1483
626 / 1483
627 / 1483
628 / 1483
629 / 1483
630 / 1483
631 / 1483
632 / 1483
633 / 1483
634 / 1483
635 / 1483
636 / 1483
637 / 1483
638 / 1483
639 / 1483
640 / 1483
641 / 1483
642 / 1483
643 / 1483
644 / 1483
645 / 1483
646 / 1483
647 / 1483
648 / 1483
649 / 1483
650 / 1483
651 / 1483
652 / 1483
653 / 1483
654 / 1483
655 / 1483
656 / 1483
657 / 1483
658 / 1483
659 / 1483
660 / 1483
661 / 1483
662 / 1483
663 / 1483
664 / 1483
665 / 1483
666 / 1483
667 / 1483
668 / 1483
669 / 1483
670 / 1483
671 / 1483
672 / 1483
673 / 1483
674 / 1483
675 / 1483
676 / 1483
677 / 1483
678 / 1483
679 / 1483
680 / 1483
681 / 1483
682 / 1483
683 / 1483
684 / 1483
685 / 1483
686 / 1483
687 / 1483
688 / 1483
689 / 1483
690 / 1483
691 / 1483
692 / 1483
693 / 1483
694 / 1483
695 / 1483
696 / 1483
697 / 1483
698 / 1483
699 / 1483
700 / 1483
701 / 1483
702 / 1483
703 / 1483
704 / 1483

1331 / 1483
1332 / 1483
1333 / 1483
1334 / 1483
1335 / 1483
1336 / 1483
1337 / 1483
1338 / 1483
1339 / 1483
1340 / 1483
1341 / 1483
1342 / 1483
1343 / 1483
1344 / 1483
1345 / 1483
1346 / 1483
1347 / 1483
1348 / 1483
1349 / 1483
1350 / 1483
1351 / 1483
1352 / 1483
1353 / 1483
1354 / 1483
1355 / 1483
1356 / 1483
1357 / 1483
1358 / 1483
1359 / 1483
1360 / 1483
1361 / 1483
1362 / 1483
1363 / 1483
1364 / 1483
1365 / 1483
1366 / 1483
1367 / 1483
1368 / 1483
1369 / 1483
1370 / 1483
1371 / 1483
1372 / 1483
1373 / 1483
1374 / 1483
1375 / 1483
1376 / 1483
1377 / 1483
1378 / 1483
1379 / 1483
1380 / 1483
1381 / 1483
1382 / 1483
1383 / 1483
1384 / 1483
1385 / 1483
1386 / 1483
1387 / 1483
1388 / 1483
1389 / 1483
1390 / 1483
1391 / 1483
1392 / 1483
1393 / 1483
1394 / 1483
1395 / 1483
1396 / 1483
1397 / 1483
1398 / 1483
1399 / 1483
1400 / 1483
1401 / 1483
1402 / 1483
1403 / 1483
1404 / 1483
1405 / 1483
1406 / 1483
1407 / 1483
1408 / 1483
1409 / 1483
1410 / 1483
1411 / 1483
1412 / 1483
1413 / 1483
1414

In [4]:
print('hello world')

hello world
